# 1) Veriyi Oku + Sütunları Doğrula + Hızlı EDA

Bu notebook'ta:
- Veriyi yükleyip sütun isimlerini kontrol edeceğiz
- Temel istatistikler ve veri kalitesi kontrolü yapacağız
- Sınıf dağılımını inceleyeceğiz


In [1]:
import pandas as pd
from pathlib import Path


## Veriyi Yükleme


In [2]:
# Dosya yolu - Dosya aslında CSV formatında
PATH = Path("../data/all_tickets_processed_improved_v3.xlsx")
df = pd.read_csv(PATH)

print(f"Veri boyutu: {df.shape}")
print(f"\nİlk 5 satır:")
df.head()


Veri boyutu: (47837, 2)

İlk 5 satır:


,Document,Topic_group
0,connection with icon icon dear please setup ic...,Hardware
1,work experience user work experience user hi w...,Access
2,requesting for meeting requesting meeting hi p...,Hardware
3,reset passwords for external accounts re expir...,Access
4,mail verification warning hi has got attached ...,Miscellaneous


## Sütun İsimleri Kontrolü


In [3]:
# Sütunların eski halini kaydedelim
text_cols = [c for c in df.columns if c.lower() in ["document","text","message","ticket","description"]]
label_cols = [c for c in df.columns if c.lower() in ["topic_group","label","category","class","topic"]]

assert len(text_cols)>=1, f"Metin sütunu bulunamadı. Var olan sütunlar: {df.columns.tolist()}"
assert len(label_cols)>=1, f"Etiket sütunu bulunamadı. Var olan sütunlar: {df.columns.tolist()}"

TEXT_COL = text_cols[0]
LABEL_COL = label_cols[0]

print(f"Metin sütunu: {TEXT_COL}")
print(f"Etiket sütunu: {LABEL_COL}")


Metin sütunu: Document
Etiket sütunu: Topic_group


In [4]:
# Sütunları yeniden adlandır ve gereksizleri sil
df = df[[TEXT_COL, LABEL_COL]].rename(columns={TEXT_COL:"text", LABEL_COL:"label"}).dropna()
df["text"] = df["text"].astype(str).str.strip()
df = df[df["text"].str.len()>0].reset_index(drop=True)

print(f"Temizlenmiş veri boyutu: {df.shape}")


Temizlenmiş veri boyutu: (47837, 2)


## Veri Kalitesi Kontrolü


In [5]:
print(df.head(3))


                                                text     label
0  connection with icon icon dear please setup ic...  Hardware
1  work experience user work experience user hi w...    Access
2  requesting for meeting requesting meeting hi p...  Hardware


In [6]:
# Null/boş değer kontrolü
print("\n=== Null/Boş Değer Kontrolü ===")
print(f"Null değer sayısı:")
print(df.isnull().sum())
print(f"\nBoş string sayısı: {(df['text'] == '').sum()}")



=== Null/Boş Değer Kontrolü ===
Null değer sayısı:
text     0
label    0
dtype: int64

Boş string sayısı: 0


In [7]:
# Metin uzunlukları
df['text_length'] = df['text'].str.len()
print("\n=== Metin Uzunluğu İstatistikleri ===")
print(df['text_length'].describe())



=== Metin Uzunluğu İstatistikleri ===
count    47837.000000
mean       291.873989
std        388.172619
min          7.000000
25%        110.000000
50%        175.000000
75%        304.000000
max       7015.000000
Name: text_length, dtype: float64


## Sınıf Dağılımı Analizi


In [8]:
print("\n=== Sınıf Dağılımı ===")
print(f"Benzersiz sınıf sayısı: {df['label'].nunique()}")
print(f"\nSınıf frekansları (ilk 10):")
print(df["label"].value_counts().head(10))



=== Sınıf Dağılımı ===
Benzersiz sınıf sayısı: 8

Sınıf frekansları (ilk 10):
label
Hardware                 13617
HR Support               10915
Access                    7125
Miscellaneous             7060
Storage                   2777
Purchase                  2464
Internal Project          2119
Administrative rights     1760
Name: count, dtype: int64


In [9]:
print("\n=== Toplam Veri ===")
print(f"Toplam örnek sayısı: {len(df)}")
print(f"Sınıflar: {sorted(df['label'].unique())}")



=== Toplam Veri ===
Toplam örnek sayısı: 47837
Sınıflar: ['Access', 'Administrative rights', 'HR Support', 'Hardware', 'Internal Project', 'Miscellaneous', 'Purchase', 'Storage']


## Kontrol Listesi

✓ Sütun isimleri doğru mu?

✓ Null / boş ve çok kısa metinler attık mı?

✓ Sınıf sayısı makul mü? (en az 5–10 sınıf varsa güzel)


## Veriyi Kaydetme (Temizlenmiş Hali)


In [10]:
# Temizlenmiş veriyi kaydet
df_clean = df[["text", "label"]].copy()
df_clean.to_csv("../data/cleaned_data.csv", index=False)
print(f"\nTemizlenmiş veri kaydedildi: {df_clean.shape}")



Temizlenmiş veri kaydedildi: (47837, 2)
